In [119]:
import warnings

warnings.simplefilter("ignore")
from dipy.reconst.dti import fractional_anisotropy, color_fa
from argparse import ArgumentParser
from scipy import ndimage
import os
import re
import numpy as np
import nibabel as nb
import sys
import matplotlib
from PIL import Image

matplotlib.use("Agg")  # very important above pyplot import
import matplotlib.pyplot as plt
import matplotlib.image as mping


def tensor2fa(tensors, tensor_name, dti, derivdir, qcdir):
    """
    outdir: location of output directory.
    fname: name of output fa map file. default is none (name created based on
    input file)
    """
    dti_data = nb.load(dti)
    affine = dti_data.get_affine()
    dti_data = dti_data.get_data()

    # create FA map
    FA = fractional_anisotropy(tensors.evals)
    FA[np.isnan(FA)] = 0

    # generate the RGB FA map
    FA = np.clip(FA, 0, 1)
    RGB = color_fa(FA, tensors.evecs)

    fname = os.path.split(tensor_name)[1].split(".")[0] + "_fa_rgb.nii.gz"
    fa = nb.Nifti1Image(np.array(255 * RGB, "uint8"), affine)
    nb.save(fa, os.path.join(derivdir, fname))

    fa_pngs(fa, fname, qcdir)


def fa_pngs(data, fname, outdir):
    """
    data: fa map
    """
    im = data.get_data()
    fig = plot_rgb(im)
    fname = os.path.split(fname)[1].split(".")[0] + ".png"
    plt.savefig(outdir + fname, format="png")


def plot_rgb(im):
    plt.rcParams.update({"axes.labelsize": "x-large", "axes.titlesize": "x-large"})

    if im.shape == (182, 218, 182):
        x = [78, 90, 100]
        y = [82, 107, 142]
        z = [88, 103, 107]
    else:
        shap = im.shape
        x = [int(shap[0] * 0.35), int(shap[0] * 0.51), int(shap[0] * 0.65)]
        y = [int(shap[1] * 0.35), int(shap[1] * 0.51), int(shap[1] * 0.65)]
        z = [int(shap[2] * 0.35), int(shap[2] * 0.51), int(shap[2] * 0.65)]
    coords = (x, y, z)

    labs = [
        "Sagittal Slice (YZ fixed)",
        "Coronal Slice (XZ fixed)",
        "Axial Slice (XY fixed)",
    ]
    var = ["X", "Y", "Z"]

    idx = 0
    for i, coord in enumerate(coords):
        for pos in coord:
            idx += 1
            ax = plt.subplot(3, 3, idx)
            ax.set_title(var[i] + " = " + str(pos))
            if i == 0:
                image = ndimage.rotate(im[pos, :, :], 90)
            elif i == 1:
                image = ndimage.rotate(im[:, pos, :], 90)
            else:
                image = im[:, :, pos]

            if idx % 3 == 1:
                ax.set_ylabel(labs[i])
                ax.yaxis.set_ticks([0, image.shape[0] / 2, image.shape[0] - 1])
                ax.xaxis.set_ticks([0, image.shape[1] / 2, image.shape[1] - 1])

            plt.imshow(image)
#             image.show()


    fig = plt.gcf()
    fig.set_size_inches(12.5, 10.5, forward=True)
    return fig

In [121]:
m1 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_0.nii.gz'
m2 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_1.nii.gz'
m3 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_2.nii.gz'
n1 = nb.load(m1)
n2 = nb.load(m2)
n3 = nb.load(m3)
im1 = n1.get_data()
im2 = n2.get_data()
im3 = n3.get_data()
fig1 = plot_rgb(im1)
rect = fig1.patch
rect.set_facecolor('red')
# mping.imsave('e:/resulting.png', fig1, cmap= 'gray')
fig1.show()
# print(type(fig1))
# print(np.array(fig1).shape)
plt.savefig('e:/brain_4.png', format="png")
# fig2 = plot_rgb(im2)
# plt.savefig('e:/brain_5.png', format="png")
# fig3 = plot_rgb(im3)
# plt.savefig('e:/brain_6.png', format="png")

In [204]:
import numpy as np
import colorsys
rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)
hsv_to_rgb = np.vectorize(colorsys.hsv_to_rgb)

def shift_hsv(arr, delta_h, delta_s, delta_v):
    r, g, b = arr.split()
    h, s, v = rgb_to_hsv(r, g, b)
#     h += delta_h
#     s += delta_s
#     v += delta_v
#     r, g, b = hsv_to_rgb(h, s, v)
    arr = Image.merge('HSV', (h, s, v))
    return arr
# arr = pic
# arr = shift_hsv(arr, 10,10,10)
# arr.show()

In [205]:
def overlay_viz(wm_input, gm_input, csf_input):
    wm_RGB = Image.open(wm_input).convert('RGB')
    gm_RGB = Image.open(gm_input).convert('RGB')
    csf_RGB = Image.open(csf_input).convert('RGB')
#     mping.imsave('e:/brain_gray.png', fig1, cmap= 'gray')
    r1, g1, b1 = wm_RGB.split()
#     print(type(r1))
#     r_arr = np.array(r1)
#     r2, g2, b2 = gm_RGB.split()
#     g_arr = np.array(g2)
#     r3, g3, b3 = csf_RGB.split()
#     b_arr = np.array(b3)
#     t, w = r_arr.shape
#     x = np.array((t,w,3), dtype = 'float')
#     x[:,:,0] = r_arr
#     x[:,:,1] = g_arr
#     x[:,:,2] = b_arr
#     pic = Image.merge('RGB', (r_arr, g_arr, b_arr))
#     pic.show()
#     g_arr = np.array(g1)
#     b_arr = np.array(b1)
#     image_arr = np.concatenate((r_arr, g_arr, b_arr))
#     result = np.concatenate((result, image_arr))

    r2, g2, b2 = gm_RGB.split()
    r3, g3, b3 = csf_RGB.split()
    pic = Image.merge('RGB', (r3, g1, b2))
    arr = wm_RGB
    arr = shift_hsv(arr, 10,10,10)
    arr.show()
#     print(pic.mode)
#     arrayoo = np.array(np.asarray(g1).astype('float'))
#     print(arrayoo)

overlay_viz('e:/brain_4.png','e:/brain_5.png','e:/brain_6.png')
#     mping.imsave('e:/resultingimg.png', arrayoo, cmap= 'gray')
#     pic.show()
#     pic.save('e:/overlay_1.png')
#     os.chdir(namer.dirs['qa']['proproc'])
#     pic.save('overlay_1.png')
#     plt.savefig(namer.dirs['qa']['proproc']+'overlay_2.png', dpi=1990)

AttributeError: 'numpy.ndarray' object has no attribute 'mode'

In [159]:
overlay_viz('e:/brain_4.png','e:/brain_5.png','e:/brain_6.png')

RGB
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]]


In [84]:
po = Image.open('e:/brain_3.png')
j, k = po.size
psss = Image.new('RGBA', po.size, (0,0,0))
psss.paste(po, (0,0,j,k), po)
psss.save('e:/brain_1.png')